<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/branch1/GetCsvFromJsonOnline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import urllib.request, json 
with urllib.request.urlopen("https://www.lemonde.fr/webservice/decodex/updates") as url: # Obtenir le fichier JSON de https://www.lemonde.fr/webservice/decodex/updates
    data = json.loads(url.read().decode())

# data.keys() #On obtient le premier dictionnaire, sites et url, contient des dictionnaire
# data["sites"].keys() #On obtient le sous-dictionnaire, chaque site, contient des tableaux

In [3]:
# print(data)
# data.keys() # Vérifie la forme que prend les fichier JSON
# data["sites"]
# data["sites"]['1980']
# data["sites"]['1980'][2]
# data["sites"].keys()
# for i in data["sites"].keys():
#   print(type(i))

# for i in data["sites"].keys():
#   print(data["sites"][i][2])

# data["urls"].keys()
# data["urls"]['the-postillon.com']

In [ ]:
import pandas as pd

dataset = []

for i in data["sites"].keys(): # On créé un tableau avec la liste de tous les sites du fichier JSON. On a leur identifiant, leur niveau de fiabilité, une description, et 2 noms dont l'un est simplifié
  # dataset.append([int(i),data["sites"][i][0], data["sites"][i][1], data["sites"][i][2], data["sites"][i][3]])
  dataset.append([int(i),data["sites"][i][0], data["sites"][i][1]])

# tableauSites = pd.DataFrame(dataset, columns=['Identifiant','Fiabilité','Description','Nom1','Nom2']) #Pour que le tableau prenne la forme d'un tableau (dataset n'étant en réalité qu'une liste)
tableauSites = pd.DataFrame(dataset, columns=['Identifiant','Fiabilité','Description'])
tableauSites

In [ ]:
import re

dt = [] #Pour déterminer toutes les lignes dans le JSON qui ne sont pas des liens twitter
compteur = 0 #On note le numéro de chaque ligne qui n'est pas un lien twitter
for i in data["urls"].keys():
  if not(re.search('twitter', i)):
    dt.append(compteur)
  compteur = compteur+1

#On met tous les liens dans df avec des tuples (identifiant, lien)
df = [(data["urls"][i], i) for i in data["urls"].keys()]

#On retire les lignes ne contenant pas de lien twitter, on utilise dt pour ça
for index in sorted(dt, reverse=True):
    del df[index]

#append à df des tuples (identifiant, "NaN") pour tous les identifiants sans lien Twitter
for site in dataset:
  if site[0] not in [item for t in df for item in t]:
    df.append((site[0], "NaN")) #Objectif : Obtenir liste users avec Nom2 et comparaison NLP grâce à Description -> "twitter.com/"+username
df.sort(reverse=True)

#On rajoute la nouvelle colonne ne contenant que les liens au tableau
tableauSites['Twitter'] = [item[1] for item in df]

tableauSites

#dans dataset remove lignes sans lien Twitter (inutiles) puis chaque ligne prendre nom exact avec lien Twitter et obtenir X tweets (200 par exemple) puis analyse de thème dessus.

In [ ]:
save_dataset = dataset

i = 0
for line in save_dataset:
  print(line + [df[i][1]])
  save_dataset[i] = line + [df[i][1]]
  i += 1

save_dataset = [x for x in save_dataset if x[3] == "NaN"] #Retire les lignes où pas de lien

save_dataset

In [6]:
# tableauSites.to_csv('json1.csv')
# !cp json1.csv "/content/drive/My Drive/PFE"

In [7]:
# 1 : site parodique
# 2 : non fiable
# 3 : site à la fiabilité douteuse
# 4 : fiable